## 基础信息及配置

In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))

sys.path.extend(['./cnn_finetune','./lib'])

from load_cifar import load_cifar10_data, load_cifar100_data

import numpy as np; print('numpy:', np.__version__)
import sklearn; print('sklearn:', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

from lib.DataGenerator import DataGenerator

# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609] on linux


Using TensorFlow backend.


numpy: 1.16.2
sklearn: 0.20.3
tensorflow 1.13.1
keras 2.2.4


In [2]:
config = tf.ConfigProto()

# cpu
# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores, allow_soft_placement=True, device_count={'CPU': 4})
    
# gpu allow_growth
config.gpu_options.allow_growth = True

# set memory limit of gpu
# config.gpu_options.per_process_gpu_memory_fraction = 0.5

set_session(tf.Session(config=config))

## 加载模型

In [3]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger


from cnn_finetune.resnet_152 import resnet152_model

In [4]:
batch_size = 16

In [5]:
def get_model(model_name):
    img_rows, img_cols = 224, 224 # Resolution of inputs
    channel = 3
    num_classes = 2019
    
    load_weights = False
    fine_tuning = False
    
    # Load our model
    model = model_name(img_rows, img_cols, channel, 
                              num_classes=num_classes, fine_tuning=fine_tuning, load_weights=load_weights)
    return model

## 运用val数据集进行验证

In [6]:
def validate_val_dataset(model):
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
    
    # load val dataset
    val_data_generator = DataGenerator(file_type='val', corp=True, batch_size=batch_size, shuffle=False)
    
    # evaluate the model
    valid_evaluate = model.evaluate_generator(val_data_generator, 
                                     steps = len(val_data_generator),
                                     workers=30, use_multiprocessing=True,
                                     verbose=1)
    print(valid_evaluate)
    return valid_evaluate

## 预测test数据集

In [12]:
def predict(model, data_generator, steps):
    predictions = model.predict_generator(data_generator, 
                                     steps = steps,
                                     workers=30, use_multiprocessing=True,
                                     verbose=1)
    filenames = data_generator.image_iterator.filenames
    
    return filenames, predictions

## 预测过程

In [8]:
!ls -l models

total 1259764
-rw-r--r-- 1 root root 127695716 May  8 08:03 denesnet_161_20190508_2.9911_0.3629.hd5
-rw-r--r-- 1 root root 127695716 May 12 12:29 denesnet_161_4.76-0.18-model.hd5
-rw-r--r-- 1 root root 127393236 May 12 12:29 denesnet_161_4.76-0.18-weights.hd5
-rw-r--r-- 1 root root  33552480 May  7 15:29 densenet121_weights_tf.h5
drwxr-xr-x 4 root root      4096 May 14 03:44 densenet161
drwxr-xr-x 3 root root      4096 May 12 04:49 densenet161_0512_val未收敛
drwxr-xr-x 3 root root      4096 May 14 19:21 densenet161_former_3_GPU
drwxr-xr-x 3 root root      4096 May  9 16:32 densenet161_train_1
drwxr-xr-x 4 root root      4096 May  8 12:40 densenet161_val
drwxr-xr-x 3 root root      4096 May 10 00:39 densenet161_val_2
drwxr-xr-x 3 root root      4096 May 10 01:21 densenet161_val_3
drwxr-xr-x 3 root root      4096 May 10 08:48 densenet161_verifiy
-rw-r--r-- 1 root root 117815848 May  7 15:29 densenet161_weights_tf.h5
-rw-r--r-- 1 root root  59218328 May  7 15:29 densenet169_weights_tf.h5
drw

In [ ]:
!ls -l models/resnet_152

### resnet 152 

In [8]:
resnet152_predict_model = get_model(resnet152_model, weights_path)

Instructions for updating:
Colocations handled automatically by placer.
load_weights False
resnet152_model: classes: 2019 fine-tuning: False
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2 (None, 230, 230, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9408        conv1_zeropadding[0][0]          
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         co

scale5a_branch1 (Scale)         (None, 7, 7, 2048)   4096        bn5a_branch1[0][0]               
__________________________________________________________________________________________________
add_4 (Add)                     (None, 7, 7, 2048)   0           scale5a_branch2c[0][0]           
                                                                 scale5a_branch1[0][0]            
__________________________________________________________________________________________________
res5a_relu (Activation)         (None, 7, 7, 2048)   0           add_4[0][0]                      
__________________________________________________________________________________________________
res5b_branch2a (Conv2D)         (None, 7, 7, 512)    1048576     res5a_relu[0][0]                 
__________________________________________________________________________________________________
bn5b_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2a[0][0]             
__________

In [ ]:
# load weight
weights_path="./models/resnet_152_copy/imaterialist-e01-1.9607.hd5"
resnet152_predict_model.load_weights(weights_path)

In [9]:
validate_val_dataset(resnet152_predict_model)

Found 10095 images belonging to 2019 classes.
631/631 [==============================] - 84s 132ms/step
[1.9606757754556845, 0.5303615651666793]


[1.9606757754556845, 0.5303615651666793]



In [15]:
val_data_generator = DataGenerator(file_type='val', corp=True, batch_size=batch_size, shuffle=False)

Found 10095 images belonging to 2019 classes.


In [16]:
rest152_filenames, rest152_pred = predict(resnet152_predict_model, val_data_generator)

631/631 [==============================] - 88s 139ms/step


In [19]:
print(rest152_pred[0])

[0.00053844 0.00045923 0.00057119 ... 0.00031146 0.0008152  0.00055016]


In [ ]:
test_data_generator = DataGenerator(file_type='predict', corp=True, batch_size=batch_size, shuffle=False)

In [ ]:
rest152_filenames, rest152_pred = predict(resnet152_predict_model, test_data_generator)

In [ ]:
np.save("rest152_pred.npy",rest152_pred)
np.save("rest152_filenames.npy",rest152_filenames)

### densenet 161 

In [8]:
from cnn_finetune.densenet161_weights_path import densenet161_model

densenet161_predict_model = get_model(densenet161_model)

Instructions for updating:
Colocations handled automatically by placer.
dense161_model: classes: 2019 fine-tuning: False
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2 (None, 230, 230, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 96) 14112       conv1_zeropadding[0][0]          
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 96) 384         conv1[0][0]           

conv5_12_x2_zeropadding (ZeroPa (None, 9, 9, 192)    0           relu5_12_x2[0][0]                
__________________________________________________________________________________________________
conv5_12_x2 (Conv2D)            (None, 7, 7, 48)     82944       conv5_12_x2_zeropadding[0][0]    
__________________________________________________________________________________________________
concat_5_12 (Concatenate)       (None, 7, 7, 1632)   0           concat_5_11[0][0]                
                                                                 conv5_12_x2[0][0]                
__________________________________________________________________________________________________
conv5_13_x1_bn (BatchNormalizat (None, 7, 7, 1632)   6528        concat_5_12[0][0]                
__________________________________________________________________________________________________
conv5_13_x1_scale (Scale)       (None, 7, 7, 1632)   3264        conv5_13_x1_bn[0][0]             
__________

In [9]:
weights_path="./models/temp_submit/imaterialist-e01-1.8070.hd5"
densenet161_predict_model.load_weights(weights_path)

In [10]:
validate_val_dataset(densenet161_predict_model)

Found 10095 images belonging to 2019 classes.
631/631 [==============================] - 104s 165ms/step
[1.806962543008586, 0.5274888559105728]


[1.806962543008586, 0.5274888559105728]

In [11]:
train_data_generator = DataGenerator(file_type='train', corp=True, batch_size=batch_size, shuffle=False)

Found 1008641 images belonging to 2019 classes.


In [13]:
densenet161_filenames, densenet161_pred = predict(densenet161_predict_model, train_data_generator, steps=len(train_data_generator)/2)

31521/31520 [==============================] - 9325s 296ms/step


In [1]:
np.save("./submissions/middle/densenet161_train_e01_1.8070_0.5275_pred_part1.npy", densenet161_pred)
np.save("./submissions/middle/densenet161_train_e01_1.8070_0.5275_filenames_part1.npy", densenet161_filenames)

NameError: name 'np' is not defined

In [17]:
densenet161_filenames = np.load("./submissions/middle/densenet161_train_e01_1.8070_0.5275_filenames_part1.npy")
densenet161_pred = np.load("./submissions/middle/densenet161_train_e01_1.8070_0.5275_pred_part1.npy")

In [30]:
densenet161_filenames = densenet161_filenames[:len(densenet161_pred)]

2018


In [31]:
densenet161_labels = train_data_generator.image_iterator.labels[:len(densenet161_pred)]

In [36]:
import numpy as np
densenet161_idx_list=[]
for idx in range(len(densenet161_pred)):
    idx_list = np.argsort(densenet161_pred[idx])
    densenet161_idx_list.append(idx_list)
    if idx % 1000 == 0:
        print(idx)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [41]:
densenet161_idx_list = np.array(densenet161_idx_list)

In [45]:
top_hit = {}
for pos in range(100):
    top_hit[pos+1] = {}
for idx in range(len(densenet161_pred)):
    if idx % 1000 == 0:
        print(idx)
    for pos in range(100):
        true_label = densenet161_labels[idx]
        if densenet161_idx_list[idx][-1-pos] == true_label:
            top_hit[pos+1][true_label] = top_hit[pos+1].get(true_label, 0) + 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [48]:
hit_count = 0
for i in range(100):
    K = i+1
    hit_count = hit_count + sum(top_hit[K].values())
    print("Top-%d:" % K)
    print(hit_count*1.0/len(densenet161_pred))
    print('-' * 20)

Top-1:
0.7384561086259954
--------------------
Top-2:
0.8240954443069699
--------------------
Top-3:
0.8562248183750516
--------------------
Top-4:
0.8742029123441515
--------------------
Top-5:
0.8862781954887218
--------------------
Top-6:
0.8954070302338124
--------------------
Top-7:
0.9022358110466039
--------------------
Top-8:
0.9078253386631134
--------------------
Top-9:
0.9124730338504489
--------------------
Top-10:
0.916446575299007
--------------------
Top-11:
0.9198133783826655
--------------------
Top-12:
0.9228093969099965
--------------------
Top-13:
0.9256646362742299
--------------------
Top-14:
0.9280677960724596
--------------------
Top-15:
0.9302350020621173
--------------------
Top-16:
0.9322733257193617
--------------------
Top-17:
0.9342382855873862
--------------------
Top-18:
0.9360089305542337
--------------------
Top-19:
0.9375594841534215
--------------------
Top-20:
0.9391080549474953
--------------------
Top-21:
0.9405019669426732
--------------------
To

## 融合模型答案

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
rest152_filenames = np.load("./submissions/middle/0522/rest152_filenames.npy")
rest152_pred = np.load("./submissions/middle/0522/rest152_pred.npy")

densenet161_filenames = np.load("./submissions/middle/0522/densenet161_filenames.npy")
densenet161_pred = np.load("./submissions/middle/0522/densenet161_pred.npy")

In [ ]:
print(len(rest152_filenames))

In [ ]:
answer = rest152_filenames == densenet161_filenames

In [ ]:
for each in answer:
    if each != True:
        print(each)

### 1. 直接相加

In [ ]:
new_pred = rest152_pred + densenet161_pred

In [ ]:
print(sum(new_pred[0]))

### 2. 取topN相加

In [ ]:
def get_topN(pred, N):
    import copy
    idx_list = []
    prediction = copy.deepcopy(pred)
    for i in range(N):
        pos = prediction.argmax()
        idx_list.append(pos)
        prediction[pos] = -1
    return idx_list

In [ ]:
def mixTopN(pred_a, pred_b, N):
    new_pred = np.zeros(2019, dtype=float)
    a_idx = get_topN(pred_a, N)
    b_idx = get_topN(pred_b, N)
    for i in range(N):
        new_pred[a_idx[i]] = new_pred[a_idx[i]] + pred_a[a_idx[i]]
        new_pred[b_idx[i]] = new_pred[b_idx[i]] + pred_b[b_idx[i]]
    return new_pred

In [ ]:
new_pred = []
for idx in range(len(rest152_filenames)):
    tmp = mixTopN(rest152_pred[idx], densenet161_pred[idx], 5)
    new_pred.append(tmp)

In [ ]:
print(sum(new_pred[0]))

## top3获取并且包装为 csv 格式

In [ ]:
import copy

In [ ]:
predictions = copy.deepcopy(new_pred)

In [ ]:
top3=[]
top3_cnt = 0
for idx in range(len(filenames)):
    predict = copy.deepcopy(predictions[idx])
    temp = []
    for j in range(3):
        pos = predict.argmax()
        temp.append(str(pos))
        predict[pos]=-1
    image_id = filenames[idx].split('/')[-1]
    top3.append([image_id, ' '.join(temp)])

In [ ]:
print(len(top3))

In [ ]:
top3.append(["e156a54a448d0e120593bbe1e50470f5.jpg", "0 1 2"])

In [ ]:
import csv
submission_filename = "./submissions/submission_0522_result_4.csv"
headers=['id', 'predicted']
result = top3
with open(submission_filename, "w", newline='') as csvfile:
    writer  = csv.writer(csvfile)
    writer.writerow(headers)
    for row in result:
        writer.writerow(row)

In [ ]:
import csv
import re

pattern = re.compile("\\d{1,}\ \\d{1,}\ \\d{1,}")  # 'int int int'的格式

check_flag = True

NUM_TEST = 90834

with open(submission_filename) as csvfile:
    print(submission_filename)
    csv_reader = csv.reader(csvfile)
    header = next(csv_reader)
    
    # judge the header
    if(header == ['id', 'predicted']):
        print("Header is right.")
    else:
        check_flag = False
        print("Wrong header:", header)
        print("Accepted header is ['id', 'predicted']")
    
    #judege each answer
    cnt = 0
    for each in csv_reader:
        cnt = cnt+1
        if len(each) != 2:
            check_flag = False
            print("Accepted result is [id, class]")
        if not pattern.match(each[-1]):
            check_flag = False
            print(cnt)
            print(each)
            print("Wrong result format:", each[-1])
            
    if(NUM_TEST != cnt):
        check_flag = False
        print("Wrong number of submission file.")
        
    if check_flag:
        print("You can submit the result")
    else:
        print("!!! Check the result again")
        